In [2]:
import requests,json,pandas

In [18]:
def getStatsQueryData(url):
    kv = {'user-agent': 'Mozilla/5.0'}
    js = json.loads(requests.get(url, headers=kv).text)
    reg = {r["code"]: r["cname"] for r in js["returndata"]["wdnodes"][1]["nodes"]}
    sj = {r["code"]: r["cname"] for r in js["returndata"]["wdnodes"][2]["nodes"]}
    v={}
    for x in js["returndata"]["datanodes"]:
        name = reg[x["wds"][1]["valuecode"]]
        code = x["wds"][1]["valuecode"]
        if name in v:
            v[name][sj[x["wds"][2]["valuecode"]]]=x["data"]["data"]
        else:
            v[name] = {"name":name,"code":code,
                       sj[x["wds"][2]["valuecode"]]:x["data"]["data"]}
    return (js["returndata"]["wdnodes"][0]["nodes"][0]["cname"],
           pandas.DataFrame([v[k] for k in v]))

In [19]:
url = "http://data.stats.gov.cn/easyquery.htm?\
m=QueryData&dbcode=fsnd&rowcode=reg&colcode=sj&\
wds=%5B%7B%22wdcode%22%3A%22zb%22%2C%22valuecode%22%3A%22\
A0K0101%22%7D%5D&dfwds=%5B%7B%22wdcode%22%3A%22sj%22%2C%22\
valuecode%22%3A%22LAST20%22%7D%5D&k1=1570764644918"

In [57]:
name,data = getStatsQueryData(url)

In [58]:
data.head()

,name,code,2018年,2017年,2016年,2015年,2014年,2013年,2012年,2011年,...,2008年,2007年,2006年,2005年,2004年,2003年,2002年,2001年,2000年,1999年
0,北京市,110000,0,5129.81,5070.00,4605.00,4608.00,4794.68,5149.00,5416.00,...,4459.13,4580,4026.3,3618.91,3173.43,1903.54,3115,2945.99,2768.00,2496
1,天津市,120000,0,3751.47,3556.87,3298.11,2992.10,2591.28,2226.41,1755.53,...,1001.39,779,625.9,509.01,412.53,329.47,342,280.17,231.76,209
2,河北省,130000,0,578.69,552.41,501.91,534.19,585.78,544.94,447.65,...,273.95,309,243.1,209.17,190.42,84.60,167,156.61,141.90,124
3,山西省,140000,0,350.14,317.38,297.10,280.73,822.68,720.24,567.19,...,300.65,222,164.2,116.22,81.23,36.27,75,59.47,49.91,43
4,内蒙古自治区,150000,0,1245.56,1139.03,962.49,1002.96,962.29,771.96,670.97,...,577.19,545,403.7,352.07,253.30,138.45,149,137.40,126.45,120


In [59]:
nc = {}
for c in data.columns:
    if "年" in c:
        nc[c] = "y{0}".format(c.split("年")[0])
data.rename(columns=nc,inplace=True) 
data = data.drop(["y2018"],axis=1)

In [ ]:
data.to_csv("./data/旅游收入.csv",index=False)